In [1]:
# Import SQLAlchemy and other dependencies here
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy import Column, Float, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base
Base2 = declarative_base()

import pandas as pd
import numpy as np

In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
database_path = "../data/employees.csv"

engine = create_engine(f"sqlite:///{database_path}")


In [4]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# Collect the names of tables within the database
inspector.get_table_names()

In [ ]:
# Using the inspector to print the column names within the 'employees' table and its types
columns = inspector.get_columns('employees')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
#create employees class
class employees(Base2):
    __tablename__ = 'employees'
    
    emp_no = Column(Integer, primary_key=True)
    birth_date = Column(Date)
    first_name = Column(String)
    last_name = Column(String)
    gender = Column(String)
    hire_date = Column(Date)

In [ ]:
session = Session(engine)

In [ ]:
# print sums by gender
male = session.query(employees).filter_by(gender = 'M').count()
female = session.query(employees).filter_by(gender = 'F').count()

print(male)
print(female)

In [ ]:
# Using the inspector to print the column names within the 'Salaries' table and its types
columns = inspector.get_columns('salaries')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
#create salaries class
class salaries(Base2):
    __tablename__ = 'salaries'
    emp_no = Column(Integer, primary_key=True)
    salary = Column(Integer)
    from_date = Column(Date)
    to_date = Column(Date)

In [ ]:
#query the salaries table
x = session.query(salaries.salary)

In [ ]:
# Plot the Results in a Matplotlib bar chart
df = pd.DataFrame(x, columns=['salary'])
a = np.array(df)
x = a[0:,0]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# the histogram of the data
n, bins, patches = plt.hist(x, 12, facecolor='purple', alpha=0.75)

plt.xlabel('Salary')
plt.ylabel('Frequency')
plt.title('Histogram of Employee Salaries')

plt.grid(True)
plt.show()
fig.savefig('Employee_Salary_Histogram.png')

In [ ]:
# Using the inspector to print the column names within the 'titles' table and its types
columns = inspector.get_columns('titles')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
#create titles class
class titles(Base2):
    __tablename__ = 'titles'
    
    emp_no = Column(Integer, primary_key=True)
    title = Column(String)
    from_date = Column(Date)
    to_date = Column(Date)

In [ ]:
# Query titles using `distinct`
session.query(titles.title).distinct().all()

In [ ]:
# Query average salaries by title
a = session.query(titles.title, func.avg(salaries.salary)).\
    filter(salaries.emp_no == titles.emp_no).\
    group_by(titles.title).\
    order_by(func.avg(salaries.salary).desc()).all()
x = np.array(a)
x

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
N = 7
title1 = x[0:,0]
avg_sal = x[0:,1]
ind = np.arange(N)    # the x locations for the groups
width = 0.5       # the width of the bars: can also be len(x) sequence

plt.bar(ind, avg_sal, width)

plt.ylabel('Avg. Salary ($)')
plt.title('Avg. Salary by Title')
plt.xticks(ind, (title1), rotation=45)

plt.show()
fig.savefig('Avg_Salary_Title.png')